In [349]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
import os


print("Import Complete")

Import Complete


In [350]:
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [351]:
def label_features(df, features_master):
        for i,row in df.iterrows():
            #message = pruning_dict.remove_nonalphanumeric(row.text)
            message = str(row.question_class)
            features = build_vocabulary([message], word_drop=False) & features_master
            features = features + features_master
            features = list(np.array(list(features.values())) - 1)
            df.set_value(i,'features',features)
        return df
    
def build_vocabulary(messages, word_drop=True):
    cvocabularies = Counter()
    #print("building vocabulary from {} messages".format(len(messages)))
    for start in range(0, len(messages), 500):
        vocabulary = Counter()
        for message in messages[start:start + 500]:
            message = str(message)
            #message = remove_nonalphanumeric(message)
            message_split = message.split()
            gram_count = 3
            grams = []
            for i in range(len(message_split) - (gram_count - 1)):
                gram = ''
                for n in range(gram_count):
                    gram = gram + message_split[i+n] + ' '
                grams.append(gram[:-1])
            vocabulary = vocabulary + Counter(message_split) + Counter(grams)
        cvocabularies = cvocabularies + vocabulary
    if word_drop == True:
        for key, count in dropwhile(
                                    lambda key_count: key_count[1] >= (len(messages) * .01),
                                    cvocabularies.most_common()):
            del cvocabularies[key]
    #print("The vocabulary contains {} words".format(len(cvocabularies)))
    return cvocabularies

In [352]:
def cluster(df, df2, N, v=False):
    #print("Start loop")
    #gpercentages = []
    #singles = 0
    clusterer = KMeans(n_clusters=N)
    clusterer.fit(df2)
    transform = clusterer.transform(df2)
    
    
    d_center = []
    cluster = []
    for x in transform:
        d_center.append(min(x)**2)
        cluster.append(np.argmin(x))
    df['cluster'] = cluster
    df['d_from_center'] = d_center
    d_center = np.array(d_center)
    mean = np.mean(d_center)
    std = np.std(d_center)

    print("Mean: {}".format(round(mean, 3)))
    print("STD: {}".format(round(std, 3)))
    print("")

  

    if v == True:
        for cgroup in range(N):
            group = df.groupby('cluster').get_group(cgroup)
            print_clusters(group)

    
    return df

def print_clusters(group):
    std = np.std(list(group.d_from_center))
    print("Found {} messages of the same form.   STD: {}".format(len(group), std))
    for message in group.question_class.head(5):
        if group.question_class.count() > 1:
            print(message)
            print("")
    print("")

In [353]:
def plot_clusters(df, N):
    features = list(df.features)
    svd = TruncatedSVD(n_components=50, n_iter=2, random_state=42)
    #svd =  PCA(n_components=2)
    tsne = TSNE(n_components=2, perplexity=10, verbose=2)
    feature_reduction =  make_pipeline(svd, tsne)

    transformed = pd.DataFrame(feature_reduction.fit_transform(features), columns=["plot_cordX", "plot_cordY"])
    meanX = transformed["plot_cordX"].mean()
    meanY = transformed["plot_cordY"].mean()
    transformed["plot_cordX"] = transformed["plot_cordX"].divide(meanX) - 1
    transformed["plot_cordY"] = transformed["plot_cordY"].divide(meanY) - 1
    meanX = transformed["plot_cordX"].mean()



    df["plot_cordX"] = transformed.plot_cordX
    df["plot_cordY"] = transformed.plot_cordY

    q = df["plot_cordX"].quantile(0.9)
    df = df[df["plot_cordX"] < q]
    q = df["plot_cordY"].quantile(0.9)
    df = df[df["plot_cordY"] < q]


    for n in range(N):
        plt.scatter(df[df["cluster"] == n].plot_cordX, df[df["cluster"] == n].plot_cordY, label="Class " + str(n))



    #plt.legend()
    plt.show()



In [354]:
csvdata = "./data/subcategory_2_think_hr_v2.csv"
df = pd.read_csv(csvdata)
df = df[['question_class']]
df = df.dropna()

N = 20




vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.7 
                                 )
X = vectorizer.fit_transform(list(df.question_class))


svd = TruncatedSVD(n_components=75)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)
df["features"] = list(X)


explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))

print(len(vectorizer.vocabulary_) )

cluster(df, X, N, v=True)

'''
#vocab = load_obj('final_feature_set')
vocab = list(vectorizer.vocabulary_)
features_master = Counter(vocab)
df["features"] = [[0] * len(vocab)] * len(df)
df = label_features(df, features_master)
'''

#plot_clusters(df, N)



Explained variance of the SVD step: 39%
497
Mean: 0.689
STD: 0.18

Found 902 messages of the same form.   STD: 0.130009324997
hello group employee turning enrolling eligible medicare part husband group plan well qualifying event husband enroll state continuation thank cassie

group offering retiree med supp plan program retirees meet certain number years service asking updating plan document type plan spd thought would retiree policy outlines plan parameters reason would want put something like place better way illustrate program

baby want add wife child plan also change plans time qle

getting mixed message deposit employee contributions simple retirement plan plan says days end month make deposit however received informational notice referring safe harbor act apparently says deposit must made within business days date payroll deduction deposit preceding entire month th next month help clarify thank

employee eligible health plan daughter years old however daughter child given adopti

'\n#vocab = load_obj(\'final_feature_set\')\nvocab = list(vectorizer.vocabulary_)\nfeatures_master = Counter(vocab)\ndf["features"] = [[0] * len(vocab)] * len(df)\ndf = label_features(df, features_master)\n'

In [355]:
  
with open('doc_tf_idf.tsv','w') as w:
    for question in X:
        string = ""
        for v in question:
            string = string + str(v) + "\t"
        w.write(string + os.linesep)
w.close
print("Done")        

with open('doc_meta.tsv','w') as w:
    w.write("cluster\tquestion\t" + os.linesep)
    for question, cluster in zip(list(df.question_class), list(df.cluster)):
        string = ""
        string = str(cluster) + "\t" + str(question) + "\t"
        w.write(string + os.linesep)       
w.close
print("Done")


Done
Done
